# Partie sur la prediction des données

## Classification des elections du second tour 2022

### Préparation des données

In [1]:
import pandas as pd
import random
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
#creation du dataset

#Presidentielle 2 tour 2022
df_resultatPresidentielle = pd.read_excel('../resultats-par-niveau-subcom-t2-france-entiere.xlsx').dropna()
df_resultatPresidentielle = df_resultatPresidentielle.iloc[:, [0,2,7, 25, 32]]
df_resultatPresidentielle.columns = ['dep','com', 'Abstentions', 'Macron', 'Lepen']
df_resultatPresidentielle['COM'] = df_resultatPresidentielle['dep'].astype(str) + df_resultatPresidentielle['com'].astype(str).str.zfill(3)
df_resultatPresidentielle = df_resultatPresidentielle.drop(['dep','com'], axis=1)
#print(df_resultatPresidentielle)

#election presidentielle 1 tour 2022
df_election2022 = pd.read_excel('../AutreDataCommune/resultats-par-niveau-burvot-t1-france-entiere.xlsx').dropna()
df_election2022 = df_election2022.iloc[:, [0,4,9, 27, 34, 41, 48, 55, 62, 69, 76, 83, 90, 97, 104]]
df_election2022.columns = ['dep','com', 'ABSTENTIONS', 'ARTHAUD', 'ROUSSEL', 'MACRON', 'LASSALLE', 'LEPEN', 'ZEMMOUR', 'MÉLENCHON', 'HIDALGO', 'JADOT', 'PÉCRESSE', 'POUTOU', 'DUPONT-AIGNAN']
df_election2022['COM'] = df_election2022['dep'].astype(str) + df_election2022['com'].astype(str).str.zfill(3)
df_election2022 = df_election2022.drop(['dep','com'], axis=1)
df_election2022 = df_election2022.groupby('COM')[['ABSTENTIONS', 'ARTHAUD', 'ROUSSEL', 'MACRON', 'LASSALLE', 'LEPEN', 'ZEMMOUR', 'MÉLENCHON', 'HIDALGO', 'JADOT', 'PÉCRESSE', 'POUTOU', 'DUPONT-AIGNAN']].mean().reset_index()
#print(df_election2022)

#Presidentielle 2 tour 2017
df_election2017 = pd.read_excel('../AutreDataCommune/Presidentielle_2017_Resultats_Communes_Tour_2_c.xls').dropna()
df_election2017 = df_election2017.iloc[1:, [0,2, 6, 24, 31]]
df_election2017.columns = ['dep','com','Abstentions17','Macron17', 'Lepen17']
df_election2017['COM'] = df_election2017['dep'].astype(str).str.zfill(2) + df_election2017['com'].astype(str).str.zfill(3)
df_election2017 = df_election2017.drop(['dep','com'], axis=1)
#print(df_election2017)

ImportError: Pandas requires version '3.1.0' or newer of 'openpyxl' (version '3.0.10' currently installed).

In [ ]:
#Niveau d'etude par sexe 
df_niveauEtude = pd.read_excel('../AutreDataCommune/diplomesFormation-2020.xlsx', sheet_name='COM_2020')
df_niveauEtude = df_niveauEtude.iloc[5:, [0,55,56,57,58,59,60,61,63,64,65,66,67,68,69]].dropna()
df_niveauEtude.columns = ['COM','P20_HNSCOL15P_DIPLMIN','P20_HNSCOL15P_BEPC','P20_HNSCOL15P_CAPBEP','P20_HNSCOL15P_BAC','P20_HNSCOL15P_SUP2','P20_HNSCOL15P_SUP34','P20_HNSCOL15P_SUP5','P20_FNSCOL15P_DIPLMIN','P20_FNSCOL15P_BEPC','P20_FNSCOL15P_CAPBEP','P20_FNSCOL15P_BAC','P20_FNSCOL15P_SUP2','P20_FNSCOL15P_SUP34','P20_FNSCOL15P_SUP5']
#print(df_niveauEtude)

#Proprotion d'imigration
df_PopImmigre = pd.read_excel('../AutreDataCommune/ImmigréCommune.xlsx', sheet_name='Data')
df_PopImmigre = df_PopImmigre[df_PopImmigre.iloc[:,2] == '2020']
df_PopImmigre = df_PopImmigre.iloc[:, [0, 3]]
df_PopImmigre.columns = ['COM', 'part_immigres']
df_PopImmigre.dropna()
#print(df_PopImmigre)

#Taux de chomage
df_chomeur = pd.read_csv('../AutreDataCommune/DemandeursEmploiInscriparCommunePlus5000hab.csv', skiprows=3, skipfooter=3, delimiter=';').dropna()
df_chomeur.columns = ['COM', 'chomeur']
df_chomeur['NCOM'] = df_chomeur['COM'].str.extract(r'([a-zA-Zéàêèôâîï\-_]+)')
df_chomeur['COM'] = df_chomeur['COM'].str.extract(r'(\d+)')
df_chomeur = df_chomeur.sort_values(by='COM')
df_chomeur['chomeur'] = df_chomeur['chomeur'].str.replace(' ', '')
df_chomeur['chomeur'] = df_chomeur['chomeur'].astype(float)
#print(df_chomeur)

#Tranche d'age
df_age = pd.read_excel('../AutreDataCommune/age.xlsx', sheet_name='COM')
df_age = df_age.iloc[:, [0,1, 12,13,14,15,16,17,23,24,25,26,27,28]]
df_age['COM'] = df_age['COM'].astype(str).str.zfill(5)
df_age.dropna()
#print(df_age)

#crime pour mille habitant
df_crime = pd.read_csv('../AutreDataDepartement/crimes_communes_2022.csv', delimiter=';')
df_crime = df_crime[df_crime.iloc[:,1] == '2020-01-01']
#print(df_crime)

#Revenue moyen par commune
df_revenue = pd.read_excel('../AutreDataCommune/RevenueCommune2013.xlsx').dropna()
df_revenue = df_revenue.iloc[:, [0, 2]]
df_revenue.columns = ['COM', 'RevenueCommune']
#print(df_revenue)

In [ ]:
#Nettoyage des données et Fusion: AGE, CRIME et CHOMAGE

#Creation du pourcentage par colonne
df_age['PopTotal'] = df_age.iloc[:, 2:].sum(axis=1)
for colonne in df_age.columns[2:-1]:
    df_age[colonne] = df_age[colonne] / df_age['PopTotal'] * 100

chomeur_dict = df_chomeur.set_index('NCOM')['chomeur'].to_dict()
df_age['chomeur'] = df_age['NCOM'].map(chomeur_dict)

#Rajout des données manquante avec le hazard et Creation du pourcentage par colonne
for index, row in df_age.iterrows():
    if row['chomeur'] * 0.4 > row['PopTotal'] or pd.isnull(row['chomeur']):
        df_age.at[index, 'chomeur'] = row['PopTotal'] * (random.uniform(5, 10)/100)
df_age['chomeur'] = df_age['chomeur'] / df_age['PopTotal'] * 100

#Nettoyage des données et Creation du pourcentage par colonne
nouvelle_colonne = df_crime.groupby('code_insee')['tauxpourmille'].sum()
df_age['crime'] = df_age['COM'].map(nouvelle_colonne)

for index, row in df_age.iterrows():
    if pd.isnull(row['crime']):
        df_age.at[index, 'crime'] = 0
df_age = df_age.drop(columns=['NCOM'])

In [ ]:
#Combinaison des données

df_concat = pd.merge(df_resultatPresidentielle, df_election2022,  on='COM')
df_concat = pd.merge(df_concat, df_election2017, on='COM')
df_concat = pd.merge(df_concat, df_niveauEtude, on='COM')
df_concat = pd.merge(df_concat, df_PopImmigre, on='COM')
df_concat = pd.merge(df_concat, df_age, on='COM')
df_concat = pd.merge(df_concat, df_revenue, on='COM')

#print(df_concat.head())
print(df_concat.info())
print(df_concat.describe())

In [ ]:
#coherence des données
import seaborn as sns

sns.histplot(data=df_concat)
plt.show()

In [ ]:
#Preparation des données pour les entrainements

# Sélection des features
features = df_concat.drop(['COM','Macron', 'Lepen', 'Abstentions'], axis=1)
y = [0 if x <= 50 else 1 for x in df_concat['Macron']]

# Normalisation des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

#division des données
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#definition des labels
target_names = ['Lepen', 'Macron']

### Application de SVM, en faisant du copier coller du tp8

In [ ]:
svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, y_train)

# Prédire sur les données de test et évaluer les performances
y_pred_linear = svm_linear.predict(X_test)
print("Performance du SVM avec un noyau linéaire :")
print(classification_report(y_test, y_pred_linear, target_names=target_names))

In [ ]:
svm_linear = SVC(kernel='poly')
svm_linear.fit(X_train, y_train)

# Prédire sur les données de test et évaluer les performances
y_pred_linear = svm_linear.predict(X_test)
print("Performance du SVM avec un noyau linéaire :")
print(classification_report(y_test, y_pred_linear, target_names=target_names))

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1]}
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_svm = grid_search.best_estimator_
y_pred_best = best_svm.predict(X_test)
print("Meilleurs paramètres pour SVM:", grid_search.best_params_)
print("Performance du SVM avec les meilleurs paramètres :")
print(classification_report(y_test, y_pred_best, target_names=target_names))

### Application de Regression

In [ ]:
# régression logistique
from sklearn.linear_model import LogisticRegression

logReg = LogisticRegression()
logReg.fit(X_train, y_train)

y_pred = logReg.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
# Classifier using Ridge regression
from sklearn.linear_model import RidgeClassifier

ridgeClf = RidgeClassifier()
ridgeClf.fit(X_train, y_train)

y_pred = ridgeClf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
from sklearn.linear_model import SGDClassifier
#Linear classifiers (SVM, logistic regression, etc.) with SGD training.

sgdClf = SGDClassifier()
sgdClf.fit(X_train, y_train)

y_pred = sgdClf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

### Application de Foret aléatoire

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Entraînement d'une forêt aléatoire
forest_model = RandomForestClassifier(n_estimators=1000, random_state=42)
forest_model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred_forest = forest_model.predict(X_test)

print(classification_report(y_test, y_pred_forest, target_names=target_names))

### Reseau de neuronne

In [ ]:
import tensorflow as tf
from tensorflow.keras import optimizers, losses, callbacks
import numpy as np

X_trainNN = np.array(X_train)
y_trainNN = np.array(y_train)
X_valNN, X_testNN = train_test_split(X_test, test_size=0.5, random_state=42)
y_valNN, y_testNN = train_test_split(y_test, test_size=0.5, random_state=42)

X_valNN = np.array(X_valNN)
y_valNN = np.array(y_valNN)

X_testNN = np.array(X_testNN)
y_testNN = np.array(y_testNN)

CallBackSave = callbacks.ModelCheckpoint("best_model_classification.keras", 
                                         monitor='val_loss', 
                                         save_best_only=True, 
                                         mode='min')

callbackStop = callbacks.EarlyStopping(monitor='val_loss', 
                                                  patience=70)

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(features.shape[1], activation='relu'),
    tf.keras.layers.Dense(int(features.shape[1]*3/4), activation='relu'),
    tf.keras.layers.Dense(int(features.shape[1]/2), activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss=losses.binary_crossentropy,
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])     

history = model.fit(X_trainNN, y_trainNN,
                    epochs=1000,validation_data=(X_valNN, y_valNN),
                    callbacks=[CallBackSave,callbackStop]
                   )


In [ ]:
best_model = tf.keras.models.load_model("best_model_classification.keras")

y_pred_NN = best_model.predict(X_testNN)
y_pred_binary = (y_pred_NN > 0.5).astype(int)
print(classification_report(y_testNN, y_pred_binary, target_names=target_names))

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = [*range(len(acc))]

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Accuracy", "Validation Accuracy"])
plt.show()

In [ ]:
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = [*range(len(loss))]

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["loss", "Validation loss"])
plt.show()

### KNN voisin

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

acc=0
bestN_neighbors=1

X_valKNN, X_testKNN = train_test_split(X_test, test_size=0.5, random_state=42)
y_valKNN, y_testKNN = train_test_split(y_test, test_size=0.5, random_state=42)


for i in range (1,X_testKNN.shape[0]+1):
    knn_model = KNeighborsClassifier(n_neighbors=i)
    knn_model.fit(X_train, y_train)
    
    y_pred = knn_model.predict(X_valKNN)
    matriceConfusion = classification_report(y_valKNN, y_pred, output_dict=True)
    
    precision_class_0 = matriceConfusion['0']['precision']
    precision_class_1 = matriceConfusion['1']['precision']
    
    avg_precision = (precision_class_0 + precision_class_1) / 2
    
    if avg_precision > acc:
        bestN_neighbors = i
        acc = avg_precision
        
knn_model = KNeighborsClassifier(n_neighbors=bestN_neighbors)
knn_model.fit(X_train, y_train)
    
y_pred = knn_model.predict(X_testKNN)
print(classification_report(y_testKNN, y_pred, target_names=target_names))

In [ ]:
from ydata_profiling import ProfileReport

rapport = ProfileReport(df_concat)
rapport.to_file("rapport_corrélations.html")